In [17]:
import sys
import os
import json
import boto3
from io import StringIO
import pandas as pd
#if argv['params']['general'].get('scenario_path_S3') contains immense s3 path (ex: uuid-123-456-789/)
params = {'general':{}}
default = {'scenario':'orchestrator2','training_folder': '../..', 'params': params} # Default execution parameters
# here sceneario: demand is only use not on lambda.
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))
print(argv)

{'scenario': 'orchestrator2', 'training_folder': '../..', 'params': {'general': {}}}


In [ ]:
training_folder = argv['training_folder']
output_folder = training_folder + '/outputs/'
#output_folder = training_folder + '/scenarios/' + argv['scenario'] + '/outputs/'

volumes = pd.read_csv(output_folder+'volumes.csv')
volumes = volumes.set_index('index')

In [15]:
if argv['params']['general'].get('scenario_path_S3'):


    s3_path = argv['params']['general'].get('scenario_path_S3')
    session = boto3.Session(region_name='eu-west-1')
    s3 = session.client('s3')
    
    csv_buffer = StringIO()
    volumes.to_csv(csv_buffer)

    s3.put_object(Body=csv_buffer.getvalue(),
                  Bucket='ir-dev-external-shared-eu-west-1-common', 
                  Key=os.path.join('quetzal',s3_path,'outputs/volumes.csv'),
                  ACL='bucket-owner-full-control' )